# How to propagate callbacks constructor

Most LangChain modules allow you to pass ```callbacks``` directly into the constructor (i.e., initializer). 


In this case, the callbacks will only be called for that instance (and any nested runs).



# ⚠️ Important Note
Constructor callbacks are scoped only to the object they are defined on. 

They are not inherited by children of the object. 

This can lead to confusing behavior, and it's generally better to pass callbacks as a run time argument.

Refer to next notebook for this.

In [7]:
from typing import Any, Dict, List

from langchain_openai import ChatOpenAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate

In [8]:
class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        print(f"Chain {serialized.get('name')} started")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")

In [9]:
callbacks = [LoggingHandler()]
llm = ChatOpenAI(model="gpt-4o-mini", callbacks=callbacks)

In [10]:
prompt = ChatPromptTemplate.from_template("What is 1 + {number}?")

In [11]:
chain = prompt | llm

In [12]:
chain.invoke({"number": "2"})

Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='1 + 2 equals 3.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='1 + 2 equals 3.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb57b463-187e-4a63-bc66-7d83537abe20-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23}))]] llm_output={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857'} run=None


AIMessage(content='1 + 2 equals 3.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb57b463-187e-4a63-bc66-7d83537abe20-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23})